## Online Banking Simulation using Python GUI and SQLite

In [1]:
# import required Libraries
from tkinter import *
import sqlite3
import time
from tkinter import messagebox

In [2]:
# Create required SQLite Database and Tables
try:
    con=sqlite3.connect(database='gebank.sqlite')
    cur=con.cursor()

    t1='create table accounts(acn integer primary key autoincrement default 100000,name text not null,pwd text not null check(length(pwd)==8),mob text not null check(length(mob)==10),email text not null check(email like "%_@_%._%"),acn_date text,bal float,acn_type text,adhr text check(length(adhr)==12))'

    t2='create table txn(acn integer,txn_type text,amt float,bal float,txn_date text)'

    cur.execute(t1)
    cur.execute(t2)

    con.commit()
    con.close()

    print('Table Created')
    
except Exception as e:
    print('Something went Wrong,',e)

Something went Wrong, table accounts already exists


In [3]:
# create GUI window
win=Tk()
win.state('zoomed')
win.resizable(width=False,height=False)
win.configure(bg='black')

# create label for Title
title=Label(win,text='Online Banking Simulation',bg='black',fg='#e3be1b',font=('arial',45,'bold','underline'))
title.pack()

# create mainscreen function and sub_functions for buttons, by clicking those buttons we jump to other screen
def main_screen():
    frm=Frame(win)
    frm.configure(bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    
    # create new frame for new screen and destroy old frame
    def new():
        frm.destroy()
        newaccount_screen()
        
    def fp():
        frm.destroy()
        forgotpass_screen()
    
    # Function for Login button with validations
    def login():
        acn=acnentry.get()
        pwd=pwdentry.get()
        # check for blanks
        if(len(acn)==0 or len(pwd)==0):
            messagebox.showwarning('Validation','Empty fields are not allowed')
        else:
            # create connection and fetch record
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('select * from accounts where acn=? and pwd=?',(acn,pwd))
            global userrow
            userrow=cur.fetchone()
            con.close()
            if(userrow==None):
                messagebox.showerror('Login','Invalid Account No. or Password')
            else:    
                frm.destroy()
                login_screen()
        
    # create label for Sub_Title
    oblbl=Label(frm,text='Online Banking Login Page---',bg='black',fg='white',font=('arial',20,'bold','underline'))
    oblbl.place(relx=.1,rely=.2)
    
    # create account no. label for login
    acnlbl=Label(frm,text='Account No.',bg='black',fg='white',font=('arial',20,'bold'))
    acnlbl.place(relx=.20,rely=.35)
    
    # create account no. text box for entry
    acnentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    acnentry.place(relx=.42,rely=.35)
    
    # create password label for login
    pwdlbl=Label(frm,text='Password',bg='black',fg='white',font=('arial',20,'bold'))
    pwdlbl.place(relx=.20,rely=.45)
    
    # create password text box for entry
    pwdentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black',show='*')
    pwdentry.place(relx=.42,rely=.45)
    
    # create Login button to jump login screen (new frame or screen)
    loginbtn=Button(frm,text='Login',font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white',command=login)
    loginbtn.place(relx=.2,rely=.57)
    
    # create forgot password button to jump forgot password screen (new frame or screen)
    fpbtn=Button(frm,text='Forgot Password',width=18,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=fp)
    fpbtn.place(relx=.4,rely=.57)
    
    # create new account button to jump new account screen (new frame or screen)
    newbtn=Button(frm,text='Open New Account',width=22,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=new)
    newbtn.place(relx=.3,rely=.70)
    
# create new account screen function   
def newaccount_screen():
    frm=Frame(win)
    frm.configure(bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    
    # function for back button
    def back():
        frm.destroy()
        main_screen()
    
    # function for register buttton with validation
    def register():
        adh=adhentry.get()
        name=nameentry.get()
        pwd=pwdentry.get()
        mob=mobentry.get()
        email=emailentry.get()
        bal=0
        acntype='Saving'
        dt=time.ctime()
        # check constraints
        if(len(adh)==0 or len(name)==0 or len(pwd)==0 or len(mob)==0 or len(email)==0):
            messagebox.showerror('Validation','Empty fields are not allowed.')
            
        elif(len(adh)!=12):
            messagebox.showerror('Validation','Aadhaar No. should be exact 12 digits.')
        
        elif(len(name)<2):
            messagebox.showerror('Validation','Name should be 2 or more alphabets.')
            
        elif(len(pwd)!=8):
            messagebox.showerror('Validation','Password should be exact 8 digits.') 
            
        elif(len(mob)!=10):
            messagebox.showerror('Validation','Mobile Number should be exact 10 digits')
            
        else:
            # create connection and insert record
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('insert into accounts(adhr,name,pwd,mob,email,acn_date,bal,acn_type) values(?,?,?,?,?,?,?,?)',(adh,name,pwd,mob,email,dt,bal,acntype))
            con.commit()
            con.close()

            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('select max(acn) from accounts')
            row=cur.fetchone()
            con.close()

            messagebox.showinfo("Success",f'Account opened with Account No.:{row[0]}')
            frm.destroy()
            main_screen()
        
    # create back button
    backbtn=Button(frm,text='back',font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white',command=back)
    backbtn.place(relx=.85,rely=0)
    
    # create aadhaar card label for new account 
    adhlbl=Label(frm,text='Aadhaar No.',bg='black',fg='white',font=('arial',20,'bold'))
    adhlbl.place(relx=.2,rely=.2)
    
    # create aadhaar card text box for entry
    adhentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    adhentry.place(relx=.42,rely=.2)
    
    # create name label for new account
    namelbl=Label(frm,text='User Name',bg='black',fg='white',font=('arial',20,'bold'))
    namelbl.place(relx=.2,rely=.32)
    
    # create name text box for entry
    nameentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    nameentry.place(relx=.42,rely=.32)
    
    # create password label for new account
    pwdlbl=Label(frm,text='Password',bg='black',fg='white',font=('arial',20,'bold'))
    pwdlbl.place(relx=.2,rely=.44)
    
    # create password text box for entry
    pwdentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    pwdentry.place(relx=.42,rely=.44)
    
    # create mobile label for new account
    moblbl=Label(frm,text='Mobile No.',bg='black',fg='white',font=('arial',20,'bold'))
    moblbl.place(relx=.2,rely=.56)
    
    # create mobile text box for entry
    mobentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    mobentry.place(relx=.42,rely=.56)
    
    # create email label for new account
    emaillbl=Label(frm,text='Email ID',bg='black',fg='white',font=('arial',20,'bold'))
    emaillbl.place(relx=.2,rely=.68)
    
    # create email text box for entry
    emailentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    emailentry.place(relx=.42,rely=.68)
    
    # create register button 
    regbtn=Button(frm,text='Register',font=('arial',20,'bold'),bd=5,bg='black',fg='white',width=15,command=register)
    regbtn.place(relx=.4,rely=.80)

    
# create function for forgot password screen
def forgotpass_screen():
    frm=Frame(win)
    frm.configure(bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    
    # create back button function
    def back():
        frm.destroy()
        main_screen()
    
    # create password recover function
    def recover():
        adh=adhentry.get()
        acn=acnentry.get()
        mob=mobentry.get()
        email=emailentry.get()
        # check for blanks
        if(len(adh)==0 or len(acn)==0 or len(mob)==0 or len(email)==0):
            messagebox.showerror('Validation','Empty fields are not allowed')
        # create connection and fetch record if found create password
        con=sqlite3.connect(database='gebank.sqlite')
        cur=con.cursor()
        cur.execute('select pwd from accounts where adhr=? and acn=? and mob=? and email=?',(adh,acn,mob,email))
        row=cur.fetchone()
        con.close()
        if(row==None):
            messagebox.showerror('Fail','Details are not correct')
            
            frm.destroy()
            main_screen()
        else:
            messagebox.showinfo('Success',f'Your password is {row[0]}')
            
            frm.destroy()
            main_screen()
        
    # create back button
    backbtn=Button(frm,text='Back',font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white',command=back)
    backbtn.place(relx=.85,rely=0)
    
    # create aadhaar label for forgot password
    adhlbl=Label(frm,text='Aadhaar No.',bg='black',fg='white',font=('arial',20,'bold'))
    adhlbl.place(relx=.2,rely=.2)
    
    # create aadhaar text bor for entry
    adhentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    adhentry.place(relx=.42,rely=.2)
    
    # create account no. label for forgot password
    acnlbl=Label(frm,text='Account No.',bg='black',fg='white',font=('arial',20,'bold'))
    acnlbl.place(relx=.2,rely=.32)
    
    # create account no. text box for entry
    acnentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    acnentry.place(relx=.42,rely=.32)
    
    # create mobile label for forgot password
    moblbl=Label(frm,text='Mobile No.',bg='black',fg='white',font=('arial',20,'bold'))
    moblbl.place(relx=.2,rely=.44)
    
    # create mobile text box for entry
    mobentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,fg='white',bg='black')
    mobentry.place(relx=.42,rely=.44)
    
    # create email label for forgot password
    emaillbl=Label(frm,text='Email ID',bg='black',fg='white',font=('arial',20,'bold'))
    emaillbl.place(relx=.2,rely=.56)
    
    # create email text box for entry
    emailentry=Entry(frm,font=('arial',20,'bold'),bd=5,width=20,bg='black',fg='white')
    emailentry.place(relx=.42,rely=.56)
    
    # create recover or view password button
    recoverbtn=Button(frm,text='View Password',font=('arial',20,'bold'),bd=5,width=20,bg='black',fg='white',command=recover)
    recoverbtn.place(relx=.35,rely=.7)
    
# create funcion for login screen    
def login_screen():
    frm=Frame(win)
    frm.configure(bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    
    # create logout button function 
    def logout():
        frm.destroy()
        main_screen()
        
    # create check balance button function with mini frame   
    def bal():
        # create mini frame
        ifrm=Frame(frm)
        ifrm.configure(bg='black',highlightthickness=3,highlightbackground='white')
        ifrm.place(relx=.3,rely=.2,relwidth=.5,relheight=.5)
        # create connection and fetch record
        con=sqlite3.connect(database='gebank.sqlite')
        cur=con.cursor()
        cur.execute('select acn,acn_date,bal from accounts where acn=?',(userrow[0],))
        row=cur.fetchone()
        con.close()
        Label(ifrm,text=f'ACN:\t{row[0]}',font=('arial',20),bg='black',fg='white').place(relx=.05,rely=.1)
        Label(ifrm,text=f'Bal:\t{row[2]}',font=('arial',20),bg='black',fg='white').place(relx=.05,rely=.25)
        Label(ifrm,text=f'Open Date:\t{row[1]}',font=('arial',20),bg='black',fg='white').place(relx=.05,rely=.4)
     
    # create transaction button function with miniframe
    def txn():
        # create mini frame
        ifrm=Frame(frm)
        ifrm.configure(bg='black',highlightthickness=3,highlightbackground='white')
        ifrm.place(relx=.3,rely=.2,relwidth=.5,relheight=.5)
        # create connection and fetch record
        con=sqlite3.connect(database='gebank.sqlite')
        cur=con.cursor()
        cur.execute('select * from txn where acn=?',(userrow[0],))
        rows=cur.fetchall()
        con.close()
        # use loop to fetch txn.records one by one
        y=.1
        for row in rows:    
            Label(ifrm,text=row[1:],bg='black',fg='white',font=('arial',15)).place(relx=.05,rely=y)
            y+=.1
        
    # create deposit button function with miniframe 
    def deposit():
        # create mini frame
        ifrm=Frame(frm)
        ifrm.configure(bg='black',highlightthickness=3,highlightbackground='white')
        ifrm.place(relx=.3,rely=.2,relwidth=.5,relheight=.5)
        # create deposit submit button
        def dep():
            amt=float(entry.get())
            # create connection and update data
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('update accounts set bal=bal+? where acn=?',(amt,userrow[0]))
            con.commit()
            con.close()
            # create connection and fetch new bal
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('select bal from accounts where acn=?',(userrow[0],))
            bal=cur.fetchone()[0]
            con.close()
            # create connection and insert into txn table
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('insert into txn values(?,?,?,?,?)',(userrow[0],'Cr.',amt,bal,time.ctime()))
            con.commit()
            con.close()
            messagebox.showinfo('Success','Amount deposited')
            
        # create label for amount
        lbl=Label(ifrm,text='Amount',font=('arial',20,'bold'),bg='black',fg='white')
        lbl.place(relx=.2,rely=.15)
        # create entry box for amount
        entry=Entry(ifrm,font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white')
        entry.place(relx=.4,rely=.15)
        # create submit button
        btn=Button(ifrm,text='Submit',font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=dep)
        btn.place(relx=.4,rely=.45)
        
    # create withdrawl button function with miniframe 
    def withdraw():
        # create mini frame
        ifrm=Frame(frm)
        ifrm.configure(bg='black',highlightthickness=3,highlightbackground='white')
        ifrm.place(relx=.3,rely=.2,relwidth=.5,relheight=.5)
        
        # create withdraw submit button function
        def draw():
            amt=float(entry.get())
            # create connection and fetch record
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('select bal from accounts where acn=?',(userrow[0],))
            bal=cur.fetchone()[0]
            con.close()
            # check bal and withdraw amount
            if(bal>amt):
                # create connection and update bal
                con=sqlite3.connect(database='gebank.sqlite')
                cur=con.cursor()
                cur.execute('update accounts set bal=bal-? where acn=?',(amt,userrow[0]))
                con.commit()
                con.close()
                # create connection and insert into txn table
                con=sqlite3.connect(database='gebank.sqlite')
                cur=con.cursor()
                cur.execute('insert into txn values(?,?,?,?,?)',(userrow[0],'Db.',amt,bal-amt,time.ctime()))
                con.commit()
                con.close()
                messagebox.showinfo('Success','Amount withdrawn')
            else:
                messagebox.showwarning('Fail','Insufficient Balance')
        # create lable for amount
        lbl=Label(ifrm,text='Amount',font=('arial',20,'bold'),bg='black',fg='white')
        lbl.place(relx=.2,rely=.15)
        # create entry box for amount
        entry=Entry(ifrm,font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white')
        entry.place(relx=.4,rely=.15)
        # create submit button
        btn=Button(ifrm,text='Submit',font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=draw)
        btn.place(relx=.4,rely=.45)
    
    # create detail update function with miniframe
    def update():
        # create mini frame
        ifrm=Frame(frm)
        ifrm.configure(bg='black',highlightthickness=3,highlightbackground='white')
        ifrm.place(relx=.3,rely=.2,relwidth=.5,relheight=.5)
        # create update function    
        def up():
            name=nameentry.get()
            pwd=pwdentry.get()
            mob=mobentry.get()
            email=emailentry.get()
            # create connection and update details
            con=sqlite3.connect(database='gebank.sqlite')
            cur=con.cursor()
            cur.execute('update accounts set name=?,pwd=?,mob=?,email=? where acn=?',(name,pwd,mob,email,userrow[0]))
            con.commit()
            con.close()
            messagebox.showinfo('Success','Record updated')
            
        
        # create name label
        namelbl=Label(ifrm,text='User Name',bg='black',fg='white',font=('arial',15,'bold'))
        namelbl.place(relx=.1,rely=.15)
        # create entry box for name
        nameentry=Entry(ifrm,font=('arial',15),bd=5,width=15,fg='white',bg='black')
        nameentry.place(relx=.4,rely=.15)
        nameentry.insert(0,userrow[1])
        # create lable for password
        pwdlbl=Label(ifrm,text='Password',bg='black',fg='white',font=('arial',15,'bold'))
        pwdlbl.place(relx=.1,rely=.27)
        # create entry box for password
        pwdentry=Entry(ifrm,font=('arial',15),bd=5,width=15,fg='white',bg='black')
        pwdentry.place(relx=.4,rely=.27)
        pwdentry.insert(0,userrow[2])
        # create mobile label
        moblbl=Label(ifrm,text='Mobile No.',bg='black',fg='white',font=('arial',15,'bold'))
        moblbl.place(relx=.1,rely=.4)
        # create entry box for mobile
        mobentry=Entry(ifrm,font=('arial',15),bd=5,width=15,fg='white',bg='black')
        mobentry.place(relx=.4,rely=.4)
        mobentry.insert(0,userrow[3])

        # create email label
        emaillbl=Label(ifrm,text='Email ID',bg='black',fg='white',font=('arial',15,'bold'))
        emaillbl.place(relx=.1,rely=.52)
        # create entry box for email
        emailentry=Entry(ifrm,font=('arial',15),bd=5,width=15,fg='white',bg='black')
        emailentry.place(relx=.4,rely=.52)
        emailentry.insert(0,userrow[4])
        # create update button
        upbtn=Button(ifrm,text='Update',font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=up)
        upbtn.place(relx=.1,rely=.7)
        
    
    # create welcome label with user name
    wellbl=Label(frm,text=f'Welcome,{userrow[1]}',bg='black',fg='white',font=('arial',25,'bold'))
    wellbl.place(relx=0,rely=0)
    
    # create logout button
    logoutbtn=Button(frm,text='Logout',font=('arial',20,'bold'),bd=5,width=12,bg='black',fg='white',command=logout)
    logoutbtn.place(relx=.85,rely=0)
    
    # create check balance button
    balbtn=Button(frm,text='Check Balance',width=12,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=bal)
    balbtn.place(relx=0,rely=.2)
    
    # create deposit button
    depbtn=Button(frm,text='Deposit',width=12,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=deposit)
    depbtn.place(relx=0,rely=.35)
    
    # create withdraw button
    withbtn=Button(frm,text='Withdraw',width=12,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=withdraw)
    withbtn.place(relx=0,rely=.5)
    
    # create transaction button
    txnbtn=Button(frm,text='Transaction',width=12,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=txn)
    txnbtn.place(relx=0,rely=.65)
    
    # create balance update button
    updatebtn=Button(frm,text='Update',width=12,font=('arial',20,'bold'),bd=5,bg='black',fg='white',command=update)
    updatebtn.place(relx=0,rely=.8)
    
# call main screen function   
main_screen()    

# call window for GUI
win.mainloop()

In [4]:
# check all Account details
con=sqlite3.connect(database='gebank.sqlite')
cur=con.cursor()
cur.execute('select * from accounts')
for tup in cur:
    print(tup)
con.close()

(100001, 'Glenn Dowson', 'glenn333', '1231231230', 'glenn@gmail.c', 'Wed Oct 19 22:41:27 2022', 10500.0, 'Saving', '112233445566')
(100002, 'Jack Smith', 'jack3333', '1471471470', 'jack@gmail.c', 'Wed Oct 19 21:38:12 2022', 13000.0, 'Saving', '121222223232')
(100003, 'Lucy Will', 'lucy3333', '1020304050', 'lucy@gmail.c', 'Wed Oct 19 22:53:05 2022', 14000.0, 'Saving', '101010101010')
(100004, 'John Dwen', 'John3333', '3030303030', 'john@gmail.c', 'Tue Dec 20 14:03:53 2022', 8100.0, 'Saving', '212132323131')
